In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/SkinFormer"

/content/drive/MyDrive/SkinFormer


In [ ]:
%cd "/content/drive/MyDrive/UNSW/SkinFormer"


/content/drive/.shortcut-targets-by-id/1jbMn8Vv-NOdyZv5qQRqYRYo-QlNIhF9S/SkinFormer


In [3]:
!pip install -q einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.6 MB/s eta 0:00:00


In [ ]:
!python Train_DeiT-B_LRP.py --config configs.yml

In [25]:
!python DisTranPrune.py --config configs.yml

CUDA is available: True 

Starting... 

{'train': 12809, 'val': 3203}
{'train': 12809, 'val': 3203}
Weights load from /content/drive/MyDrive/SkinFormer/Outputs/LRP/DeiT-S_LRP_checkpoint_BASE.pth
Weights load from /content/drive/MyDrive/SkinFormer/Outputs/LRP/DeiT-S_LRP_checkpoint_6SA_detection.pth
+++++++++++++++++++++++++++++ Pruning Iteration 0 +++++++++++++++++++++++++++++
Generating masks: 100% 100/100 [01:27<00:00,  1.14it/s]
threshold=6.672273400454287e-09
number of params allowed to be pruned 7762.0
number of params pruned in head 0: 3104.0/38809
threshold=5.9555835818514424e-09
number of params allowed to be pruned 7762.0
number of params pruned in head 1: 3104.0/38809
threshold=4.6785769702850644e-11
number of params allowed to be pruned 7762.0
number of params pruned in head 2: 3104.0/38809
threshold=2.9754712027596497e-09
number of params allowed to be pruned 7762.0
number of params pruned in head 3: 3104.0/38809
threshold=4.577877188438606e-09
number of params allowed to be

# Playground

In [ ]:
import torch

In [ ]:
SA_attr = torch.rand((5, 5))
SA_attr

tensor([[0.9466, 0.8329, 0.6962, 0.1073, 0.2920],
        [0.5274, 0.2614, 0.5630, 0.3543, 0.8221],
        [0.4174, 0.4107, 0.9575, 0.0620, 0.7003],
        [0.5561, 0.0107, 0.9163, 0.8507, 0.3080],
        [0.4083, 0.6926, 0.8765, 0.2823, 0.4795]])

In [ ]:
main_attr = torch.rand((5, 5))
main_attr

tensor([[0.0462, 0.4286, 0.3788, 0.0762, 0.4209],
        [0.4656, 0.3276, 0.7632, 0.1814, 0.9515],
        [0.7088, 0.6977, 0.6378, 0.0540, 0.6413],
        [0.7743, 0.6409, 0.3244, 0.7367, 0.8930],
        [0.7998, 0.4699, 0.8183, 0.7289, 0.1575]])

In [ ]:
percentile = 80


flat_tensor = main_attr.flatten()

threshold = torch.quantile(flat_tensor, 0.2)

mask1 = (main_attr < threshold).float()
mask1

tensor([[1., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.]])

In [ ]:
can_be_pruned = SA_attr * mask1
can_be_pruned
can_be_pruned_flt = can_be_pruned.flatten()

k = int(0.6*mask1.sum())
top_k_values, top_k_indices = torch.topk(can_be_pruned_flt, k)
mask2 = torch.ones_like(can_be_pruned_flt)
mask2[top_k_indices] = 0
mask2 = mask2.reshape((5, 5))
mask2

tensor([[0., 1., 1., 1., 1.],
        [1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 0.]])

In [15]:
import torch

In [19]:
a= torch.rand((6, 197, 197))
b=torch.rand((64, 6, 197, 197))


In [20]:
a.shape

torch.Size([6, 197, 197])

In [21]:
a=a.expand(b.shape)

In [23]:
a.shape

torch.Size([64, 6, 197, 197])

In [24]:
torch.equal(a[0], a[1])

True